In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import statsmodels
import matplotlib.pyplot as plt
import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('combined_features.csv',index_col=0)

In [3]:
data

,vwap,SMA(5),SMA(10),12dayEWM,rsi,MACD,mom,mfi,spread
time,,,,,,,,,
2021-10-04 08:00:00,1.659676,4.768259,4.786339,4.771101,1.107056,0.867368,0.543952,43.744151,102.684738
2021-10-04 09:00:00,1.659675,4.766889,4.776943,4.769641,1.032303,0.822687,0.844880,42.875897,102.612395
2021-10-04 10:00:00,1.660783,4.769485,4.775439,4.779400,1.121594,0.918496,2.310488,42.838238,104.369796
2021-10-04 11:00:00,1.664391,4.774242,4.771703,4.777858,1.000573,0.913061,0.528564,24.307401,103.491902
2021-10-04 12:00:00,1.664180,4.777072,4.769049,4.770852,0.963529,0.904516,0.586155,11.604788,103.568593
...,...,...,...,...,...,...,...,...,...
2022-09-23 13:00:00,1.579399,4.908354,4.900728,4.905692,1.341466,-0.357752,-2.098000,-26.144407,97.885000
2022-09-23 14:00:00,1.579414,4.905133,4.904967,4.903628,1.290696,-0.426738,-2.333600,-34.984305,97.701400
2022-09-23 15:00:00,1.579429,4.894920,4.903911,4.890514,1.061713,-0.552944,-3.514100,-43.456076,96.735900


In [4]:
def gen_labels(df,t,threshold):
    # Calculate % return on spread t hours later
    fw_ret = df['spread'].diff(periods=t)/df['spread']
    
    #If the return is more than x%, we should have bought, and hence the label is (1)
    #If return is less than x%, we should have sold, and hence label is (-1)
    #If in between, do nothing (0)
    output = np.select([fw_ret > threshold ,fw_ret < -threshold],[1,-1])
    return output

In [5]:
X = data.copy()
X = (X-X.mean())/(X.max()-X.min())
X['spread'] = data['spread']
X

,vwap,SMA(5),SMA(10),12dayEWM,rsi,MACD,mom,mfi,spread
time,,,,,,,,,
2021-10-04 08:00:00,0.744148,0.503772,0.520310,0.507095,0.020827,0.150135,0.024185,0.246218,102.684738
2021-10-04 09:00:00,0.744142,0.503058,0.515350,0.506332,-0.004853,0.142435,0.036776,0.241323,102.612395
2021-10-04 10:00:00,0.754814,0.504410,0.514556,0.511432,0.025821,0.158945,0.098097,0.241111,104.369796
2021-10-04 11:00:00,0.789572,0.506886,0.512584,0.510626,-0.015754,0.158008,0.023541,0.136632,103.491902
2021-10-04 12:00:00,0.787540,0.508360,0.511182,0.506965,-0.028480,0.156536,0.025950,0.065013,103.568593
...,...,...,...,...,...,...,...,...,...
2022-09-23 13:00:00,-0.029126,0.576715,0.580694,0.577422,0.101355,-0.060974,-0.086355,-0.147821,97.885000
2022-09-23 14:00:00,-0.028974,0.575038,0.582932,0.576344,0.083914,-0.072862,-0.096213,-0.197662,97.701400
2022-09-23 15:00:00,-0.028836,0.569720,0.582375,0.569492,0.005250,-0.094609,-0.145605,-0.245426,96.735900


In [6]:
y = gen_labels(X,t=24,threshold=0.02)

In [7]:
np.unique(y,return_counts = True)

(array([-1,  0,  1]), array([ 570, 1665,  559], dtype=int64))

In [8]:
split = int(len(data) * 0.8)

X_train = X[:split]
y_train = y[:split]

X_test = X[split:]
y_test = y[split:]

### Preprocessing: Time Series Split
We do not have a lot of data to work with, and hence will need to cross-validate.

In [9]:
#Game plan: shit out the model (wuds the initial train test tho?), gridsearchcv until it doesn't suck, display final metrics, export the model as pkl
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV, TimeSeriesSplit

In [10]:
tscv = TimeSeriesSplit()
clf = SVC(decision_function_shape='ovo',class_weight='balanced')

In [11]:
cross_val_score(clf, X_train, y_train, cv=tscv, scoring='f1_weighted')

array([0.11020641, 0.30987267, 0.17718946, 0.23227264, 0.07962673])

In [12]:
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [13]:
finder = GridSearchCV(clf, params_grid, cv=5, scoring='f1_weighted')
finder.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=SVC(class_weight='balanced',
                           decision_function_shape='ovo'),
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}],
             scoring='f1_weighted')

In [14]:
finder.best_params_

{'C': 100, 'kernel': 'linear'}

In [15]:
finder.best_score_

0.7346316776830925

In [16]:
y_pred = finder.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.73      0.94      0.82       103
           0       0.89      0.83      0.86       380
           1       0.59      0.57      0.58        76

    accuracy                           0.81       559
   macro avg       0.74      0.78      0.75       559
weighted avg       0.82      0.81      0.81       559



In [17]:
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(finder, f)